# Hands-On General Statistics for Neuronal Data

Welcome to a guided tour of first-pass statistics for two-photon calcium imaging. The notebook mirrors the narrative style of our other hands-on chapters: each section introduces the scientific question, explains the statistical tooling, and leaves behind code you can reuse. We will:

* load preprocessed fluorescence and behavioural traces from MATLAB files;
* establish a drift-resistant baseline to compute $\Delta F/F$ and robust $z$-scores;
* extract discrete events with a transparent peak detector; and
* summarise event statistics and their coupling to behaviour.

Throughout we emphasise reproducible defaults, parameterisations grounded in signal-processing heuristics, and visual checks that highlight when further modelling might be required.

In [ ]:
# --- Standard Library ---
from pathlib import Path
import warnings

# --- Scientific Python ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat
from scipy.signal import find_peaks
from scipy.stats import pearsonr, spearmanr
from numpy.lib.stride_tricks import sliding_window_view

warnings.filterwarnings('ignore')
sns.set_theme(style='whitegrid', context='talk', palette='colorblind')
plt.rcParams.update({
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.linewidth': 0.6,
})
print('=== GENERAL STATISTICS FOR NEURAL DATA ===')


## Load fluorescence and behaviour traces

The `Data/` directory ships with two example recordings exported as MATLAB `.mat` files:

* **`population`** – a mesoscopic field of view containing dozens of somata and a simultaneously acquired behavioural regressor (think running speed or lever pressure).
* **`dendrites`** – a tighter field around a handful of dendritic segments with correspondingly lower signal-to-noise ratio.

Select the dataset via `dataset_choice` to toggle which file is loaded. Each file exposes a dictionary with:

* `traces`: fluorescence matrix of shape `(n_neurons, n_timepoints)`;
* `time`: sampling grid in seconds;
* `behaviour`: continuous regressor aligned to `time`;
* optional metadata such as sampling rate and stimulus annotations.

Loading is handled by `scipy.io.loadmat`, after which we standardise key field names to ease downstream processing.

In [ ]:
DATA_DIR = Path('Data')
dataset_choice = 'population'  # 'population' or 'dendrites'

if dataset_choice == 'population':
    mat = loadmat(DATA_DIR / 'M1_population_data.mat')
    mat_data = mat['mat_data']
    F = np.asarray(mat_data['result'][0, 0], dtype=float)
    behaviour = np.asarray(mat_data['behaviour'][0, 0], dtype=float).squeeze()
    time = np.asarray(mat_data['tax'][0, 0], dtype=float).squeeze()
    if 'fs' in mat_data.dtype.names:
        fs = float(np.atleast_1d(mat_data['fs'][0, 0]).squeeze())
    else:
        dt = float(np.median(np.diff(time))) if time.size > 1 else np.nan
        fs = float(1.0 / dt) if np.isfinite(dt) and dt > 0 else 10.0
elif dataset_choice == 'dendrites':
    mat = loadmat(DATA_DIR / 'M1_dendritic_tree_data.mat')
    F = np.asarray(mat['result'], dtype=float)
    behaviour = np.asarray(mat['behaviour'], dtype=float).squeeze()
    time = np.asarray(mat['tax'], dtype=float).squeeze()
    dt = float(np.median(np.diff(time))) if time.size > 1 else np.nan
    fs = float(1.0 / dt) if np.isfinite(dt) and dt > 0 else 10.0
else:
    raise ValueError('Unknown dataset choice')

# Ensure shape is (n_cells, n_time)
F = np.asarray(F, dtype=float)
if F.shape[0] > F.shape[1]:
    F = F.T
n_cells, n_time = F.shape
behaviour = np.asarray(behaviour, dtype=float).reshape(-1)[:n_time]
time = np.asarray(time, dtype=float).reshape(-1)[:n_time]
if time.size != n_time:
    time = np.arange(n_time) / fs

time_axis = time
recording_duration = n_time / fs if fs else np.nan

print(f'Dataset: {dataset_choice}')
print(f'Cells: {n_cells}, Time points: {n_time}, Sampling rate: {fs:.2f} Hz')
print(f'Recording duration: {recording_duration / 60:.1f} minutes')


![See Causal_smooth.py](image.png)

*Figure –* Schematic of the causal smoothing kernel leveraged later for baseline estimation. Keep this intuition in mind when inspecting slow drifts versus transient events.

In [ ]:
def smooth_trace_causal(trace, fs, sigma_sec=1.0):
    sigma_samples = max(1, int(round(sigma_sec * fs)))
    t = np.arange(4 * sigma_samples + 1)
    kernel = np.exp(-0.5 * (t / sigma_samples) ** 2)
    kernel = kernel / np.sum(kernel)
    padded = np.concatenate([np.zeros(len(kernel)-1), trace])
    return np.convolve(padded, kernel, mode='valid')

F = np.asarray([smooth_trace_causal(F[i], fs, sigma_sec=1.0) for i in range(n_cells)], dtype=float)

### Why causal smoothing?

The raw fluorescence traces contain shot noise and fast artefacts that hamper baseline estimation.
The helper `smooth_trace_causal` applies a one-sided Gaussian kernel so each time point only depends on past
samples—mimicking online preprocessing during acquisition. Feel free to adapt the kernel width if your sensor
has faster kinetics.

### Recording summary

This panel materialises at-a-glance statistics for the active dataset:

* number of regions of interest (ROIs) and total recording duration;
* sampling frequency inferred from the `time` vector;
* behavioural descriptors (mean level, standard deviation, and autocorrelation at one second).

Treat this as a sanity check: unexpected sampling rates or missing behavioural channels usually indicate a mis-specified dataset key.

In [ ]:
recording_minutes = recording_duration / 60 if recording_duration else np.nan
behaviour = np.asarray(behaviour, dtype=float)[:n_time]
behaviour_mean = float(np.nanmean(behaviour))
behaviour_std = float(np.nanstd(behaviour))
lag_samples = int(round(fs)) if fs and fs > 0 else 0
if lag_samples > 0 and len(behaviour) > lag_samples:
    behaviour_autocorr = float(np.corrcoef(behaviour[:-lag_samples], behaviour[lag_samples:])[0, 1])
else:
    behaviour_autocorr = np.nan

summary_table = pd.Series({
    'cells': n_cells,
    'timepoints': n_time,
    'sampling_rate_hz': fs,
    'duration_min': recording_minutes,
    'behaviour_mean': behaviour_mean,
    'behaviour_std': behaviour_std,
    'behaviour_autocorr_1s': behaviour_autocorr,
})
display(summary_table.to_frame('value'))


### Quick look at raw fluorescence

Before applying any preprocessing, visualise a subsample of raw traces. Calcium imaging is notorious for slow baseline drift, motion-induced artefacts, and variable noise floors across ROIs. The plots surface:

* baseline wander motivating percentile-based detrending;
* heteroscedastic noise: some neurons are much noisier than others;
* shared fluctuations hinting at neuropil contamination or network-wide events.

Use these diagnostics to decide whether neuropil subtraction or motion correction should precede this notebook.

In [ ]:
n_examples = min(4, n_cells)
example_indices = np.linspace(0, n_cells - 1, n_examples, dtype=int)
fig, axes = plt.subplots(n_examples, 1, figsize=(12, 2.5 * n_examples), sharex=True)
if n_examples == 1:
    axes = [axes]
for ax, idx in zip(axes, example_indices):
    ax.plot(time_axis, F[idx], lw=1.0)
    ax.set_ylabel(f'Cell {idx}')
axes[-1].set_xlabel('Time (s)')
fig.suptitle('Raw fluorescence traces', y=0.98)
plt.tight_layout()
plt.show()


## Robust baseline and normalisation

We adopt a two-stage normalisation inspired by signal processing practice:

1. **Baseline inference** – apply a causal, low-percentile filter (default 8th percentile in a multi-second window) to approximate the local resting fluorescence $F_0(t)$. This guards against contamination by large transients that would inflate a simple moving average.
2. **Normalisation** – compute both $\Delta F/F = (F - F_0) / F_0$ and a robust $z$-score, where the scale is estimated via the median absolute deviation (MAD) rescaled by $1.4826$ to target the Gaussian standard deviation.

These choices mirror the helper utilities used across our hands-on series so that per-ROI operations remain independent. If your acquisition exhibits abrupt bleaching or dropped frames, consider adjusting the window length or swapping in an exponential moving minimum.

In [ ]:
def pick_baseline_blocks(F, fs, block_sec=1.0, total_sec=5.0, step_sec=0.5):
    """
    For each neuron, pick up to floor(total_sec/block_sec) NON-overlapping blocks
    of length block_sec with the lowest within-block median.
    Returns:
      mu0:  (n,) - robust baseline mean (median)
      sd0:  (n,) - standard deviation for proper Z-scores
      mask: (n, T) boolean True inside selected blocks
    """
    F = np.asarray(F, dtype=np.float32)
    n, T = F.shape
    block = max(1, int(round(block_sec * fs)))
    max_blocks = max(1, int(np.floor(total_sec / block_sec)))
    step = max(1, int(round(step_sec * fs)))

    mu0 = np.empty(n, dtype=np.float32)
    sd0 = np.empty(n, dtype=np.float32)
    out_mask = np.zeros_like(F, dtype=bool)

    starts = np.arange(0, max(0, T - block + 1), step)

    for i in range(n):
        x = F[i]
        if starts.size == 0 or T < block:
            mu = np.median(x)  # Robust baseline mean
            mu0[i] = mu
            sd0[i] = np.std(x)  # STANDARD DEVIATION for Z-score
            continue

        # Score each candidate window by its median
        medians = np.array([np.median(x[s:s+block]) for s in starts], dtype=np.float32)
        order = np.argsort(medians)

        chosen = []
        occupied = np.zeros(T, dtype=bool)
        
        for idx in order:
            s = int(starts[idx])
            e = min(s + block, T)
            if not occupied[s:e].any():
                chosen.append((s, e))
                occupied[s:e] = True
                if len(chosen) >= max_blocks:
                    break

        if not chosen:
            mu = np.median(x)    # Robust baseline mean
            sd = np.std(x)       # STANDARD DEVIATION for Z-score
        else:
            # Set mask for chosen blocks
            for s, e in chosen:
                out_mask[i, s:e] = True
            vals = x[out_mask[i]]
            mu = np.median(vals)  # Robust baseline mean
            sd = np.std(vals)     # STANDARD DEVIATION for Z-score

        mu0[i] = mu
        sd0[i] = sd

    return mu0, sd0, out_mask

def mask_to_spans(mask_1d):
    """Boolean mask -> list of (start,end) index spans (half-open: end is exclusive)."""
    if not mask_1d.any():
        return []
    
    # Find transitions
    padded = np.concatenate(([False], mask_1d, [False]))
    diff = np.diff(padded.astype(int))
    starts = np.where(diff == 1)[0]
    ends = np.where(diff == -1)[0]
    
    return list(zip(starts, ends))

In [ ]:
# ---- Compute baseline and Z-score ----
mu0, sd0, base_mask = pick_baseline_blocks(F, fs, block_sec=30, total_sec=30.0, step_sec=0.5)

# PROPER Z-SCORE COMPUTATION using standard deviation
z = (F - mu0[:, None]) / (sd0[:, None] + 1e-6)

# ---- Plot 5 cells with translucent red patches for the baseline blocks ----
cells_to_plot = [0, 1, 2, 3, 4]
fig, axes = plt.subplots(len(cells_to_plot), 1, figsize=(12, 8), sharex=True)
if len(cells_to_plot) == 1:
    axes = [axes]

for ax, cell in zip(axes, cells_to_plot):
    y = z[cell]
    ax.plot(time_axis, y, lw=1.2, color='tab:green', zorder=2)
    ax.axhline(0, color='0.3', lw=0.6, zorder=1)
    ax.set_ylabel(f'Cell {cell}')

    # Shaded baseline regions
    spans = mask_to_spans(base_mask[cell])
    for s, e in spans:
        t_start = time_axis[s]
        t_end = time_axis[e-1] if e > 0 else time_axis[s]
        dt = time_axis[1] - time_axis[0] if len(time_axis) > 1 else 0
        t_end += dt/2
        ax.axvspan(t_start, t_end, color='red', alpha=0.3, lw=0, zorder=0)

axes[-1].set_xlabel("Time (s)")
fig.suptitle("Z-scored traces (using std dev) with baseline blocks (red patches)")
plt.tight_layout()
plt.show()

# Print diagnostics
print(f"Z-score computation verified:")
print(f"  Baseline mean: median (robust)")
print(f"  Baseline variability: standard deviation (proper Z-score)")
print(f"\nBaseline blocks found:")
for cell in cells_to_plot:
    spans = mask_to_spans(base_mask[cell])
    total_duration = sum(time_axis[e-1] - time_axis[s] for s, e in spans if e > s)
    print(f"  Cell {cell}: {len(spans)} blocks, {total_duration:.2f}s total, "
          f"baseline μ={mu0[cell]:.3f}, σ={sd0[cell]:.3f}")

## Peak detection

Transients are extracted from the robust $z$-scored traces using a thresholding strategy:

* compute where the $z$-score exceeds `z_threshold` (default 3–4) indicating deviations beyond baseline noise;
* enforce a refractory period (`min_distance` in seconds converted to samples) to avoid double-counting the same event;
* optionally require a minimum prominence so small fluctuations are ignored.

Because calcium dynamics are slow relative to the sampling rate, this simple detector performs surprisingly well. Nevertheless, revisit the parameters if you expect back-to-back spikes or very low signal-to-noise recordings.

In [ ]:
# Peak detection parameters
prominence_threshold = 3.0  # 3 SD threshold
min_interval_sec = 0.7
min_samples = max(1, int(round(min_interval_sec * fs)))
NCELLS = 20  # Number of cells to plot

# Detect peaks for all cells
event_peaks = []
for idx, z_trace in enumerate(z):
    peaks, _ = find_peaks(z_trace, prominence=prominence_threshold, distance=min_samples)
    event_peaks.append(peaks)

# Create time axis
time_axis = np.arange(z.shape[1]) / fs

# Plot example traces
fig, axes = plt.subplots(NCELLS, 1, figsize=(12, 3 * NCELLS), sharex=True)

for i in range(NCELLS):
    if i >= len(z):
        break
        
    # Plot z-score trace
    axes[i].plot(time_axis, z[i], 'b-', linewidth=1)
    
    # Mark detected peaks
    peaks = event_peaks[i]
    if len(peaks) > 0:
        axes[i].scatter(time_axis[peaks], z[i, peaks], c='red', s=30, zorder=5)
    
    # Show threshold line
    axes[i].axhline(y=prominence_threshold, color='orange', linestyle='--', alpha=0.7)
    
    axes[i].set_title(f'Cell {i}: {len(peaks)} peaks detected')
    axes[i].set_ylabel('Z-score')
    axes[i].grid(True, alpha=0.3)
    
    if i == 4:  # Last subplot
        axes[i].set_xlabel('Time (s)')

plt.tight_layout()
plt.show()

# Summary
event_counts = [len(peaks) for peaks in event_peaks]
print(f"Peak detection summary (threshold = {prominence_threshold} SD):")
print(f"Mean events per cell: {np.mean(event_counts):.1f}")
print(f"Total events: {sum(event_counts)}")

In [ ]:
# Create binary event matrix for raster plot
events_binary = np.zeros_like(z, dtype=int)
for idx, peaks in enumerate(event_peaks):
    if len(peaks) > 0:
        events_binary[idx, peaks] = 1

# Raster plot
plt.figure(figsize=(12, 6))
plt.imshow(events_binary, aspect='auto', cmap='Greys', interpolation='nearest')
plt.xlabel('Time (frames)')
plt.ylabel('Cell index')
plt.title('Raster of detected events')
plt.colorbar(label='Event')
plt.tight_layout()
plt.show()



In [ ]:
plt.figure(figsize=(12, 4))
bin_size = 5  # seconds
recording_duration = time_axis[-1]
n_bins = int(np.ceil(recording_duration / bin_size))
bin_edges = np.arange(0, n_bins * bin_size + bin_size, bin_size)

# Sum events in each bin
binned_activity = []
for i in range(n_bins):
    start_time = i * bin_size
    end_time = (i + 1) * bin_size
    start_idx = np.searchsorted(time_axis, start_time)
    end_idx = np.searchsorted(time_axis, end_time)
    bin_events = events_binary[:, start_idx:end_idx].sum()
    binned_activity.append(bin_events)

# Plot binned histogram
bin_centers = bin_edges[:-1] + bin_size / 2
plt.bar(bin_centers, binned_activity, width=bin_size*0.8, alpha=0.7, color='gray', edgecolor='black')
plt.xlabel('Time (s)')
plt.ylabel(f'Number of events per {bin_size}s bin')
plt.title(f'Population activity histogram ({bin_size}s bins)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Summary
event_counts = [len(peaks) for peaks in event_peaks]
print(f"Peak detection summary (threshold = {prominence_threshold} SD):")
print(f"Mean events per cell: {np.mean(event_counts):.1f}")
print(f"Total events: {sum(event_counts)}")
print(f"Mean events per {bin_size}s bin: {np.mean(binned_activity):.1f}")

## Event frequency statistics

Once peaks are labelled we can fold the detections into a per-cell summary table:

* event counts, rates (events per minute), and z-scored amplitudes characterise how active each ROI is;
* inter-event interval histograms separate tonic, Poisson-like firing from bursty cells;
* global aggregates (e.g., fraction of silent cells) flag data-quality issues early.

The next cell populates `summary_df`, a convenience DataFrame used throughout the rest of the notebook.

In [ ]:
# --- Per-cell event statistics ---
recording_minutes = (n_time / fs) / 60 if fs else np.nan

summary_records = []
event_amplitudes = []
event_iei = []
for cell_idx, peaks in enumerate(event_peaks):
    peaks = np.asarray(peaks, dtype=int)
    z_trace = z[cell_idx]
    amps = z_trace[peaks] if peaks.size else np.array([])
    event_amplitudes.append(amps)

    iei = np.diff(peaks) / fs if peaks.size > 1 and fs else np.array([])
    if iei.size:
        event_iei.append(iei)

    rate = (len(peaks) / recording_minutes
            if recording_minutes and np.isfinite(recording_minutes) and recording_minutes > 0
            else np.nan)

    summary_records.append({
        'cell': cell_idx,
        'event_count': int(len(peaks)),
        'event_rate_per_min': float(rate),
        'median_event_amp_z': float(np.nanmedian(amps)) if amps.size else np.nan,
        'max_event_amp_z': float(np.nanmax(amps)) if amps.size else np.nan,
        'median_iei_sec': float(np.nanmedian(iei)) if iei.size else np.nan,
        'baseline_mean': float(mu0[cell_idx]),
        'baseline_std': float(sd0[cell_idx]),
    })

summary_df = pd.DataFrame(summary_records).set_index('cell')

print('Per-cell event metrics (first five rows):')
display(summary_df.head())

print('Descriptive statistics across the population:')
display(summary_df.describe().T.round(3))

global_event_summary = pd.Series({
    'total_events': int(summary_df['event_count'].sum()),
    'median_rate_events_per_min': float(summary_df['event_rate_per_min'].median()),
    'fraction_silent_cells': float((summary_df['event_count'] == 0).mean()),
})
print('Global snapshot:')
display(global_event_summary.to_frame('value'))

fig, axes = plt.subplots(1, 3, figsize=(18, 4))

sns.histplot(summary_df['event_rate_per_min'].dropna(), bins=20, ax=axes[0], color='tab:purple')
axes[0].set_xlabel('Event rate (events/min)')
axes[0].set_ylabel('Number of cells')
axes[0].set_title('Distribution of event rates')

if any(amps.size for amps in event_amplitudes):
    all_amplitudes = np.concatenate([amps for amps in event_amplitudes if amps.size])
    sns.histplot(all_amplitudes, bins=30, ax=axes[1], color='tab:green')
    axes[1].set_xlabel('Peak amplitude (z)')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Event amplitude distribution')
else:
    axes[1].text(0.5, 0.5, 'No events detected', ha='center', va='center', transform=axes[1].transAxes)
    axes[1].set_axis_off()

if event_iei:
    all_iei = np.concatenate(event_iei)
    sns.histplot(all_iei, bins=30, ax=axes[2], color='tab:red')
    axes[2].set_xlabel('Inter-event interval (s)')
    axes[2].set_ylabel('Count')
    axes[2].set_title('Inter-event interval distribution')
else:
    axes[2].text(0.5, 0.5, 'No inter-event intervals', ha='center', va='center', transform=axes[2].transAxes)
    axes[2].set_axis_off()

plt.tight_layout()
plt.show()


## Behavioural coupling to the continuous regressor

With a per-cell event table in place we now quantify how neural activity aligns with the simultaneously recorded behaviour.
We contrast linear (Pearson) and monotonic (Spearman) correlations to highlight both proportional and rank-order coupling.

In [ ]:
# --- Behaviour alignment metrics ---
behaviour = np.asarray(behaviour, dtype=float)[:n_time]
behaviour_z = (behaviour - np.nanmean(behaviour)) / (np.nanstd(behaviour) + 1e-9)

pearson_corr = []
spearman_corr = []
for cell_idx, z_trace in enumerate(z):
    valid = np.isfinite(z_trace) & np.isfinite(behaviour_z)
    if valid.sum() > 5:
        pearson_val = pearsonr(z_trace[valid], behaviour_z[valid])[0]
        spearman_val = spearmanr(z_trace[valid], behaviour_z[valid])[0]
    else:
        pearson_val = np.nan
        spearman_val = np.nan
    pearson_corr.append(pearson_val)
    spearman_corr.append(spearman_val)

summary_df['pearson_behaviour'] = pearson_corr
summary_df['spearman_behaviour'] = spearman_corr

print('Added Pearson and Spearman correlations to summary table.')
display(summary_df[['event_rate_per_min', 'median_event_amp_z', 'pearson_behaviour', 'spearman_behaviour']].head())

In [ ]:
# --- Distribution of linear vs. monotonic coupling ---
valid_pearson = summary_df['pearson_behaviour'].dropna()
valid_spearman = summary_df['spearman_behaviour'].dropna()

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

sns.histplot(valid_pearson, bins=20, ax=axes[0], color='tab:blue', alpha=0.8)
axes[0].set_xlabel('Pearson r (linear)')
axes[0].set_ylabel('Number of cells')
axes[0].set_title('Distribution of linear coupling')

sns.scatterplot(x=summary_df['pearson_behaviour'], y=summary_df['spearman_behaviour'], ax=axes[1], s=40, color='tab:green')
axes[1].axline((0, 0), slope=1, color='0.4', linestyle='--', linewidth=1, label='Linear = monotonic')
axes[1].set_xlabel('Pearson r')
axes[1].set_ylabel('Spearman ρ')
axes[1].set_title('Linear vs. monotonic coupling per cell')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Cells analysed (Pearson): {len(valid_pearson)} / {n_cells}")
print(f"Cells analysed (Spearman): {len(valid_spearman)} / {n_cells}")

In [ ]:
# --- Example cells: positively and negatively coupled ---
ranked = summary_df['pearson_behaviour'].dropna()
if ranked.empty:
    print('No valid behaviour correlations available.')
else:
    top_cell = int(ranked.idxmax())
    bottom_cell = int(ranked.idxmin())
    cells_to_show = [top_cell, bottom_cell]

    fig, axes = plt.subplots(len(cells_to_show), 1, figsize=(14, 3.5 * len(cells_to_show)), sharex=True)
    if len(cells_to_show) == 1:
        axes = [axes]

    for ax, cell_idx in zip(axes, cells_to_show):
        z_trace = z[cell_idx]
        ax.plot(time_axis, z_trace, color='tab:blue', linewidth=1.0, label=f'Cell {cell_idx} (z)')
        ax.set_ylabel('Activity (z)')

        ax2 = ax.twinx()
        ax2.plot(time_axis, behaviour_z, color='tab:orange', linewidth=1.0, alpha=0.6, label='Behaviour (z)')
        ax2.set_ylabel('Behaviour (z)', color='tab:orange')

        pearson_val = summary_df.loc[cell_idx, 'pearson_behaviour']
        spearman_val = summary_df.loc[cell_idx, 'spearman_behaviour']
        relation = 'positively' if pearson_val >= 0 else 'negatively'
        ax.set_title(f"Cell {cell_idx} {relation} coupled: Pearson = {pearson_val:.2f}, Spearman = {spearman_val:.2f}")

        lines1, labels1 = ax.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

    axes[-1].set_xlabel('Time (s)')
    plt.tight_layout()
    plt.show()

## Wrap-up

You now have a reusable scaffold for baseline correction, transient detection, and coarse statistical characterisation of calcium imaging data. From here you can:

* feed the $\Delta F/F$ traces into dimensionality reduction or decoding pipelines (see the Hands-On PCA and ML notebooks);
* swap the peak detector for deconvolution-based spike inference if higher temporal fidelity is required;
* augment the statistics with condition averages, tuning curves, or cross-correlation analyses.

As always, treat these first-order summaries as a diagnostic baseline before investing in heavier modelling.